In [1]:
!pip3 install langchain==0.0.348 openai weaviate-client ragas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 51.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
import os
import numpy as np

In [5]:
from langchain.chat_models import ChatOpenAI
from google.colab import userdata

api_key = userdata.get("open_ai")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

query = "What did the president say about Justice Breyer"


llm.predict(query)

"I'm sorry, I cannot provide real-time information as my responses are generated based on a mixture of licensed data, data created by human trainers, and publicly available data. The most recent statement made by the president about Justice Breyer may not be included in my training data."

# Step 1: Collect and load data
The first step is to identify the key sources of data you want to leverage.
To index our knowledge base, we first need to load the data.

Specify a `DocumentLoader` to load in your unstructured data as `Documents`.

A `Document` is a dict with text (`page_content`) and `metadata`.

Since we have a curated list of URLs, we can use one of Langchain’s many built-in data loaders— WebBaseLoader.  link to other loaders

In [6]:
import requests
from langchain.document_loaders import TextLoader

url = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
    f.write(res.text)


loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

documents

[Document(page_content='Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  \n\nLast year COVID-19 kept us apart. This year we are finally together again. \n\nTonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. \n\nWith a duty to one another to the American people to the Constitution. \n\nAnd with an unwavering resolve that freedom will always triumph over tyranny. \n\nSix days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. \n\nHe thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. \n\nHe met the Ukrainian people. \n\nFrom President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. \n\nGroups of citize

# Step 2: Chunk your Document
Split the `Document` into chunks for embedding and vector storage.

It’s important to chunk the data as we want to embed a meaningful length of context within our vector index.

Embedding just a word or two is too little information to match relevant vectors, and embedding entire pages would be too long to fit within the context window of the prompt. Try to strike the right balance for your use case and dataset.

There are many text splitters that Langchain supports.


Here we use...
We also set a small overlap length so that text continuity is preserved between our chunks.

In [7]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(documents)

# Step 3: Embed and store chunks
To be able to look up our document splits, we first need to store them where we can later look them up.

The most common way to do this is to embed the contents of each document split.

We store the embedding and splits in a vectorstore.

For this demo, I used Weaviate as the vector database, and OpenAI for the Embeddings.

Once our vector store is indexed, it’s time to define our retriever. Retriever is the module that determines how the relevant documents are fetched from the vector database, determined by its search algorithm.

In [8]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Weaviate
import weaviate
from weaviate.embedded import EmbeddedOptions

client = weaviate.Client(
  embedded_options = EmbeddedOptions()
)

vectorstore = Weaviate.from_documents(
    client = client,
    documents = chunks,
    embedding = OpenAIEmbeddings(),
    by_text = False)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

Binary /root/.cache/weaviate-embedded did not exist. Downloading binary from https://github.com/weaviate/weaviate/releases/download/v1.23.7/weaviate-v1.23.7-Linux-amd64.tar.gz
Started /root/.cache/weaviate-embedded: process ID 882


# Step 4: Prompt
As shown above, we can load prompts (e.g., this RAG prompt) from the prompt hub.

The prompt can also be easily customized, as shown below.

In [9]:
from langchain import PromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use three sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context","question"]
  )

print(prompt)

input_variables=['context', 'question'] template="You are an assistant for question-answering tasks. \nUse the following pieces of retrieved context to answer the question. \nIf you don't know the answer, just say that you don't know. \nUse three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:\n"


# Step 5: Generate
Distill the retrieved documents into an answer using an LLM/Chat model (e.g., `gpt-3.5-turbo`).

We use the Runnable protocol to define the chain.

Runnable protocol pipes together components in a transparent way.

We used a prompt for RAG that is checked into the LangChain prompt hub (here).

In [10]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [11]:
# Ragas wants ['question', 'answer', 'contexts', 'ground_truths'] as
'''
{
    "question": [], <-- question from faq doc
    "answer": [], <-- answer from generated result
    "contexts": [], <-- context
    "ground_truths": [] <-- actual answer
}
'''

from datasets import Dataset

questions = ["What did the president say about Justice Breyer?",
             "What did the president say about Intel's CEO?",
             "What did the president say about gun violence?",
            ]
ground_truths = [["The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service."],
                ["The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion."],
                ["The president asked Congress to pass proven measures to reduce gun violence."]]
answers = []
contexts = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    contexts.append([docs.page_content for docs in retriever.get_relevant_documents(query)])

# To dict
data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)


In [12]:
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_relevancy,
    context_recall,
    context_precision,
)

result = evaluate(
    dataset = dataset,
    metrics=[
        context_precision,
        context_recall,
        faithfulness,
        answer_relevancy,
    ],
)

result

evaluating with [context_precision]


100%|██████████| 1/1 [00:02<00:00,  2.66s/it]


evaluating with [context_recall]


100%|██████████| 1/1 [00:02<00:00,  2.78s/it]


evaluating with [faithfulness]


100%|██████████| 1/1 [00:11<00:00, 11.66s/it]


evaluating with [answer_relevancy]


100%|██████████| 1/1 [00:02<00:00,  2.05s/it]


{'context_precision': 0.3333, 'context_recall': 1.0000, 'faithfulness': 0.7222, 'answer_relevancy': 0.8544}

In [13]:
import pandas as pd
pd.set_option("display.max_colwidth", None)

df = result.to_pandas()
df

,question,answer,contexts,ground_truths,context_precision,context_recall,faithfulness,answer_relevancy
0,What did the president say about Justice Breyer?,The president honored Justice Breyer for his service and dedication to the country. He mentioned nominating Judge Ketanji Brown Jackson to continue Justice Breyer's legacy of excellence. The president did not provide any specific comments about Justice Breyer's work or character.,"[Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court., And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence., A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a broad range of support—from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \n\nAnd if we are to advance liberty and justice, we need to secure the Border and fix the immigration system.]",[The president said that Justice Breyer has dedicated his life to serve the country and thanked him for his service.],0.0,1.0,0.666667,0.807217
1,What did the president say about Intel's CEO?,"The president mentioned that Intel's CEO, Pat Gelsinger, is ready to increase their investment from $20 billion to $100 billion, which would be one of the biggest investments in American manufacturing history. The president emphasized that passing a bill would allow Intel to proceed with their investment plans. The president expressed support for Intel and highlighted the potential impact on job creation and technological advancements.","[But that’s just the beginning. \n\nIntel’s CEO, Pat Gelsinger, who is here tonight, told me they are ready to increase their investment from \n$20 billion to $100 billion. \n\nThat would be one of the biggest investments in manufacturing in American history. \n\nAnd all they’re waiting for is for you to pass this bill. \n\nSo let’s not wait any longer. Send it to my desk. I’ll sign it. \n\nAnd we will really take off. \n\nAnd Intel is not alone. \n\nThere’s something happening in America., This is where Intel, the American company that helped build Silicon Valley, is going to build its $20 billion semiconductor “mega site”. \n\nUp to eight state-of-the-art factories in one place. 10,000 new good-paying jobs. \n\nSome of the most sophisticated manufacturing in the world to make computer chips the size of a fingertip that power the world and our everyday lives. \n\nSmartphones. The Internet. Technology we have yet to invent. \n\nBut that’s just the beginning., As I said last year, especially to our younger transgender Americans, I will always have your back as your President, so you can be yourself and reach your God-given potential. \n\nWhile it often appears that we never agree, that isn’t true. I signed 80 bipartisan bills into law last year. From preventing government shutdowns to protecting Asian-Americans from still-too-common hate crimes to reforming military justice.]",[The president said that Pat Gelsinger is ready to increase Intel's investment to $100 billion.],0.0,1.0,1.000000,0.840909
2,What did the president say about gun violence?,"The president called on Congress to pass measures to reduce gun violence, including universal background checks and a ban on assault weapons and high-capacity magazines. He also mentioned repealing the liability shield for gun manufacturers to allow them to be sued. The president emphasized that these laws do not infringe on the 

We can make the following observations:

* context_precision (signal-to-noise ratio of the retrieved context): While the LLM judges all of the context as relevant for the last question, it also judges that most of the retrieved context for the second question is irrelevant. Depending on this metric, you could experiment with different numbers of retrieved contexts to reduce the noise.
* context_recall (if all the relevant information required to answer the question was retrieved): The LLMs evaluate that the retrieved contexts contain the relevant information required to answer the questions correctly.
* faithfulness (factual accuracy of the generated answer): While the LLM judges that the first and last questions are answered correctly, the answer to the second question, which wrongly states that the president did not mention Intel’s CEO, is judged with a faithfulness of 0.5.
* answer_relevancy (how relevant is the generated answer to the question): All of the generated answers are judged as fairly relevant to the questions.